In [35]:
import pandas as pd
import numpy as np

# Gerar dataset para cada estado

In [36]:
datasetTotal = pd.read_csv('datasetTotal.csv')

In [37]:
listaUF = datasetTotal['UF'].unique()

In [38]:
datasetTotal.head(10)

,Unnamed: 0,Município_x,UF,Idade,Total_Estado,Total_Homem,Total_Mulher,Urbana_Total,Urbana_Homem,Urbana_Mulher,Rural_Total,Rural_Homem,Rural_Mulher,editMun,ID,Município_y
0,0,Alta Floresta D'Oeste (RO),RO,Total,6880,3520,3360,4011,1951,2060,2869,1569,1300,altaflorestadoeste,1,Alta Floresta dOeste
1,1,NaN,RO,15 a 17 anos,1577,830,747,878,438,440,699,392,307,altaflorestadoeste,1,Alta Floresta dOeste
2,2,NaN,RO,18 ou 19 anos,1011,532,479,604,301,303,407,231,176,altaflorestadoeste,1,Alta Floresta dOeste
3,3,NaN,RO,20 a 24 anos,2200,1105,1095,1303,632,671,897,473,424,altaflorestadoeste,1,Alta Floresta dOeste
4,4,NaN,RO,25 a 29 anos,2092,1053,1039,1226,580,646,866,473,393,altaflorestadoeste,1,Alta Floresta dOeste
5,5,Ariquemes (RO),RO,Total,26480,13096,13384,23011,11284,11727,3469,1812,1657,ariquemes,2,Ariquemes
6,6,NaN,RO,15 a 17 anos,5497,2758,2739,4641,2310,2331,856,448,408,ariquemes,2,Ariquemes
7,7,NaN,RO,18 ou 19 anos,3544,1721,1823,3096,1477,1619,448,244,204,ariquemes,2,Ariquemes
8,8,NaN,RO,20 a 24 anos,8981,4456,4525,7920,3908,4012,1061,548,513,ariquemes,2,Ariquemes
9,9,NaN,RO,25 a 29 anos,8458,4161,4297,7354,3589,3765,1104,572,532,ariquemes,2,Ariquemes


In [39]:
datasetDict = {}
for i in listaUF:
  datasetDict[i] = datasetTotal.loc[(datasetTotal['UF']==i) & (datasetTotal['Idade']=='Total')]

Alguns municípios não possuem todos os dados. Esses dados são mostrados como zero.

In [40]:
datasetTotal.loc[(datasetTotal['UF']=='CE') & (datasetTotal['ID']==947)]

,Unnamed: 0,Município_x,UF,Idade,Total_Estado,Total_Homem,Total_Mulher,Urbana_Total,Urbana_Homem,Urbana_Mulher,Rural_Total,Rural_Homem,Rural_Mulher,editMun,ID,Município_y
4715,4715,Eusébio (CE),CE,Total,13839,6814,7025,13839,6814,7025,0,0,0,eusebio,947,Eusébio
4716,4716,NaN,CE,15 a 17 anos,2871,1423,1448,2871,1423,1448,0,0,0,eusebio,947,Eusébio
4717,4717,NaN,CE,18 ou 19 anos,1750,875,875,1750,875,875,0,0,0,eusebio,947,Eusébio
4718,4718,NaN,CE,20 a 24 anos,4812,2367,2445,4812,2367,2445,0,0,0,eusebio,947,Eusébio
4719,4719,NaN,CE,25 a 29 anos,4406,2149,2257,4406,2149,2257,0,0,0,eusebio,947,Eusébio


In [41]:
nova_coluna = []
name_id = "40"     #População Jovem Rural: Total
year_id = "9"      #Ano: 2010
for index, row in datasetDict['CE'].iterrows():
  region_id = datasetDict['CE'].loc[index,'ID']
  jhi_value = datasetDict['CE'].loc[index,'Rural_Total']
  if jhi_value == 0:
    print('nulo')
  else:
    sql = "INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`)  VALUES ('NULL','" + str(region_id) + "','" + name_id + "','" + year_id + "','" + str(jhi_value) + "');"
    nova_coluna.append(sql)

nulo
nulo


Antiga função para gerar nova coluna com sqls, tratando os nulos

In [9]:
#def gerar_SQL(name, name_id, year_id, datasetUF):
#  #name = 'Rural_Total'
#  #name_id = "40"     #População Jovem Rural: Total
#  #year_id = "9"      #Ano: 2010
#  nova_coluna = []
#  string = ""
#  count = 0
#  for index, row in datasetUF.iterrows():
#    region_id = datasetUF.loc[index,'ID']
#    jhi_value = datasetUF.loc[index,name]
#    if jhi_value == "-":
#      count += 1
#    else:
#      sql = "INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`)  VALUES (NULL,'" + str(region_id) + "','" + name_id + #"','" + year_id + "','" + str(jhi_value) + "');"
#      nova_coluna.append(sql)
#
#  print("O estado " + datasetUF.loc[index,'UF'] + " tem " + str(count) + " valores nulos para o indicador " + name)
#  return nova_coluna

Nova função para gerar lista de SQLs, tratando os nulos. Gera apenas o que vem depois do VALUES

In [42]:
def gerar_values(name, name_id, year_id, datasetUF):
    #name = 'Rural_Total'
    #name_id = "40"     #População Jovem Rural: Total
    #year_id = "9"      #Ano: 2010
    nova_coluna = []
    count = 0
    
    for index, row in datasetUF.iterrows():
        region_id = datasetUF.loc[index,'ID']
        jhi_value = datasetUF.loc[index,name]
        if jhi_value == 0:
            count += 1
        else:
            #sql = "INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`)  VALUES (NULL,'" + str(region_id) + "','" + name_id + "','" + year_id + "','" + str(jhi_value) + "');"
            sql = "(NULL,'" + str(region_id) + "','" + name_id + "','" + year_id + "','" + str(jhi_value) + "')"
            nova_coluna.append(sql)        

    print("O estado " + datasetUF.loc[index,'UF'] + " tem " + str(count) + " valores nulos para o indicador " + name)
    return nova_coluna

Nova função para gerar lista de SQL completo, aqui os nulos já estão tratados. Gera um insert into para cada 10 values

In [43]:
def gerar_SQL(lista):
    nova_coluna = []
    auxStr = ""
    string = "INSERT INTO `MuniEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`) VALUES "
    count = 1 #contador para cada 10
    
    #nova_coluna.append(string)
    auxStr += string
    
    for item in lista:
        if (count%10 == 0):
            #sql = item + "; \n"
            #nova_coluna.append(sql)
            #nova_coluna.append(string) 
            
            auxStr += item + "; "
            nova_coluna.append(auxStr) 
            auxStr = string
            
        else:
            auxStr += item + ", "
            #nova_coluna.append(sql)
        count +=1

    if auxStr.endswith(', '):
        auxStr = auxStr[:-2]
        auxStr += "; "
        nova_coluna.append(auxStr) 
    
    return nova_coluna

In [44]:
name = 'Total_Estado'
name_id = "3"     #População Jovem Rural: Total
year_id = "9"      #Ano: 2010

In [45]:
teste = gerar_values(name, name_id, year_id, datasetDict['AC'])

O estado AC tem 0 valores nulos para o indicador Total_Estado


In [46]:
teste

["(NULL,'53','3','9','3365')",
 "(NULL,'54','3','9','1748')",
 "(NULL,'55','3','9','6051')",
 "(NULL,'56','3','9','2239')",
 "(NULL,'57','3','9','2185')",
 "(NULL,'58','3','9','23650')",
 "(NULL,'59','3','9','4336')",
 "(NULL,'60','3','9','9168')",
 "(NULL,'61','3','9','1828')",
 "(NULL,'62','3','9','4414')",
 "(NULL,'63','3','9','2236')",
 "(NULL,'64','3','9','4177')",
 "(NULL,'65','3','9','4505')",
 "(NULL,'66','3','9','2399')",
 "(NULL,'67','3','9','101646')",
 "(NULL,'68','3','9','3977')",
 "(NULL,'69','3','9','1389')",
 "(NULL,'70','3','9','5548')",
 "(NULL,'71','3','9','11539')",
 "(NULL,'72','3','9','10044')",
 "(NULL,'73','3','9','4447')",
 "(NULL,'74','3','9','3845')"]

In [15]:
a = gerar_SQL(teste)
for i in a:
    print(i)

INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`) VALUES (NULL,'53','3','9','3365'), (NULL,'54','3','9','1748'), (NULL,'55','3','9','6051'), (NULL,'56','3','9','2239'), (NULL,'57','3','9','2185'), (NULL,'58','3','9','23650'), (NULL,'59','3','9','4336'), (NULL,'60','3','9','9168'), (NULL,'61','3','9','1828'), (NULL,'62','3','9','4414'); 
INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`) VALUES (NULL,'63','3','9','2236'), (NULL,'64','3','9','4177'), (NULL,'65','3','9','4505'), (NULL,'66','3','9','2399'), (NULL,'67','3','9','101646'), (NULL,'68','3','9','3977'), (NULL,'69','3','9','1389'), (NULL,'70','3','9','5548'), (NULL,'71','3','9','11539'), (NULL,'72','3','9','10044'); 
INSERT INTO `RelationMunicipioEstado` (`id`,`region_id`,`name_id`,`year_id`,`jhi_value`) VALUES (NULL,'73','3','9','4447'), (NULL,'74','3','9','3845'); 


In [47]:
listaUF

array(['RO', 'AC', 'AM', 'RR', 'PA', 'AP', 'TO', 'MA', 'PI', 'CE', 'RN',
       'PB', 'PE', 'AL', 'SE', 'BA', 'MG', 'ES', 'RJ', 'SP', 'PR', 'SC',
       'RS', 'MS', 'MT', 'GO', 'DF'], dtype=object)

In [15]:
name = 'Total_Estado'
name_id = "3"   # População Jovem: Total (15 a 29 anos)
year_id = "9"   # Ano: 2010

In [16]:
for item in listaUF:
    file_name = "Total_Estado/" + item +  "_Total_Estado.txt"
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    #text_file = open(file_name, "w")
    #for i in lista:
        #text_file.write(i + "\n")    
    #text_file.close()

O estado RO tem 0 valores nulos para o indicador Total_Estado
O estado AC tem 0 valores nulos para o indicador Total_Estado
O estado AM tem 0 valores nulos para o indicador Total_Estado
O estado RR tem 0 valores nulos para o indicador Total_Estado
O estado PA tem 0 valores nulos para o indicador Total_Estado
O estado AP tem 0 valores nulos para o indicador Total_Estado
O estado TO tem 0 valores nulos para o indicador Total_Estado
O estado MA tem 0 valores nulos para o indicador Total_Estado
O estado PI tem 0 valores nulos para o indicador Total_Estado
O estado CE tem 0 valores nulos para o indicador Total_Estado
O estado RN tem 0 valores nulos para o indicador Total_Estado
O estado PB tem 0 valores nulos para o indicador Total_Estado
O estado PE tem 0 valores nulos para o indicador Total_Estado
O estado AL tem 0 valores nulos para o indicador Total_Estado
O estado SE tem 0 valores nulos para o indicador Total_Estado
O estado BA tem 0 valores nulos para o indicador Total_Estado
O estado

In [48]:
file_name = "Novas/Total_Estado.sql"
text_file = open(file_name, "w")

for item in listaUF:
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    for i in lista:
        text_file.write(i + "\n")    

text_file.close()

O estado RO tem 0 valores nulos para o indicador Total_Estado
O estado AC tem 0 valores nulos para o indicador Total_Estado
O estado AM tem 0 valores nulos para o indicador Total_Estado
O estado RR tem 0 valores nulos para o indicador Total_Estado
O estado PA tem 0 valores nulos para o indicador Total_Estado
O estado AP tem 0 valores nulos para o indicador Total_Estado
O estado TO tem 0 valores nulos para o indicador Total_Estado
O estado MA tem 0 valores nulos para o indicador Total_Estado
O estado PI tem 0 valores nulos para o indicador Total_Estado
O estado CE tem 0 valores nulos para o indicador Total_Estado
O estado RN tem 0 valores nulos para o indicador Total_Estado
O estado PB tem 0 valores nulos para o indicador Total_Estado
O estado PE tem 0 valores nulos para o indicador Total_Estado
O estado AL tem 0 valores nulos para o indicador Total_Estado
O estado SE tem 0 valores nulos para o indicador Total_Estado
O estado BA tem 0 valores nulos para o indicador Total_Estado
O estado

In [49]:
name = 'Total_Homem'
name_id = "1"     #População Jovem: Homem
year_id = "9"      #Ano: 2010

In [22]:
for item in listaUF:
    file_name = "Total_Homem/" + item +  "_Total_Homem.txt"
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    #text_file = open(file_name, "w")
    #for i in lista:
    #    text_file.write(i + "\n")    
    #text_file.close()

O estado RO tem 0 valores nulos para o indicador Total_Homem
O estado AC tem 0 valores nulos para o indicador Total_Homem
O estado AM tem 0 valores nulos para o indicador Total_Homem
O estado RR tem 0 valores nulos para o indicador Total_Homem
O estado PA tem 0 valores nulos para o indicador Total_Homem
O estado AP tem 0 valores nulos para o indicador Total_Homem
O estado TO tem 0 valores nulos para o indicador Total_Homem
O estado MA tem 0 valores nulos para o indicador Total_Homem
O estado PI tem 0 valores nulos para o indicador Total_Homem
O estado CE tem 0 valores nulos para o indicador Total_Homem
O estado RN tem 0 valores nulos para o indicador Total_Homem
O estado PB tem 0 valores nulos para o indicador Total_Homem
O estado PE tem 0 valores nulos para o indicador Total_Homem
O estado AL tem 0 valores nulos para o indicador Total_Homem
O estado SE tem 0 valores nulos para o indicador Total_Homem
O estado BA tem 0 valores nulos para o indicador Total_Homem
O estado MG tem 0 valore

In [50]:
file_name = "Novas/Total_Homem.sql"
text_file = open(file_name, "w")

for item in listaUF:
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    for i in lista:
        text_file.write(i + "\n")    

text_file.close()

O estado RO tem 0 valores nulos para o indicador Total_Homem
O estado AC tem 0 valores nulos para o indicador Total_Homem
O estado AM tem 0 valores nulos para o indicador Total_Homem
O estado RR tem 0 valores nulos para o indicador Total_Homem
O estado PA tem 0 valores nulos para o indicador Total_Homem
O estado AP tem 0 valores nulos para o indicador Total_Homem
O estado TO tem 0 valores nulos para o indicador Total_Homem
O estado MA tem 0 valores nulos para o indicador Total_Homem
O estado PI tem 0 valores nulos para o indicador Total_Homem
O estado CE tem 0 valores nulos para o indicador Total_Homem
O estado RN tem 0 valores nulos para o indicador Total_Homem
O estado PB tem 0 valores nulos para o indicador Total_Homem
O estado PE tem 0 valores nulos para o indicador Total_Homem
O estado AL tem 0 valores nulos para o indicador Total_Homem
O estado SE tem 0 valores nulos para o indicador Total_Homem
O estado BA tem 0 valores nulos para o indicador Total_Homem
O estado MG tem 0 valore

In [51]:
name = 'Total_Mulher'
name_id = "2"     #População Jovem: Mulher
year_id = "9"      #Ano: 2010

In [24]:
for item in listaUF:
    file_name = "Total_Mulher/" + item +  "_Total_Mulher.txt"
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    #print(len(lista_values))
    lista = gerar_SQL(lista_values)

    #text_file = open(file_name, "w")
    #for i in lista:
    #    text_file.write(i + "\n")    
    #text_file.close()

O estado RO tem 0 valores nulos para o indicador Total_Mulher
O estado AC tem 0 valores nulos para o indicador Total_Mulher
O estado AM tem 0 valores nulos para o indicador Total_Mulher
O estado RR tem 0 valores nulos para o indicador Total_Mulher
O estado PA tem 0 valores nulos para o indicador Total_Mulher
O estado AP tem 0 valores nulos para o indicador Total_Mulher
O estado TO tem 0 valores nulos para o indicador Total_Mulher
O estado MA tem 0 valores nulos para o indicador Total_Mulher
O estado PI tem 0 valores nulos para o indicador Total_Mulher
O estado CE tem 0 valores nulos para o indicador Total_Mulher
O estado RN tem 0 valores nulos para o indicador Total_Mulher
O estado PB tem 0 valores nulos para o indicador Total_Mulher
O estado PE tem 0 valores nulos para o indicador Total_Mulher
O estado AL tem 0 valores nulos para o indicador Total_Mulher
O estado SE tem 0 valores nulos para o indicador Total_Mulher
O estado BA tem 0 valores nulos para o indicador Total_Mulher
O estado

In [52]:
file_name = "Novas/Total_Mulher.sql"
text_file = open(file_name, "w")

for item in listaUF:
    lista_values = gerar_values(name, name_id, year_id, datasetDict[item])
    lista = gerar_SQL(lista_values)

    for i in lista:
        text_file.write(i + "\n")    

text_file.close()

O estado RO tem 0 valores nulos para o indicador Total_Mulher
O estado AC tem 0 valores nulos para o indicador Total_Mulher
O estado AM tem 0 valores nulos para o indicador Total_Mulher
O estado RR tem 0 valores nulos para o indicador Total_Mulher
O estado PA tem 0 valores nulos para o indicador Total_Mulher
O estado AP tem 0 valores nulos para o indicador Total_Mulher
O estado TO tem 0 valores nulos para o indicador Total_Mulher
O estado MA tem 0 valores nulos para o indicador Total_Mulher
O estado PI tem 0 valores nulos para o indicador Total_Mulher
O estado CE tem 0 valores nulos para o indicador Total_Mulher
O estado RN tem 0 valores nulos para o indicador Total_Mulher
O estado PB tem 0 valores nulos para o indicador Total_Mulher
O estado PE tem 0 valores nulos para o indicador Total_Mulher
O estado AL tem 0 valores nulos para o indicador Total_Mulher
O estado SE tem 0 valores nulos para o indicador Total_Mulher
O estado BA tem 0 valores nulos para o indicador Total_Mulher
O estado

In [57]:
name1 = 'Total_Homem'
name2 = 'Urbana_Homem'
name = 'Rural_Homem'
    
for item in listaUF:
    print(item)
    for index, row in datasetDict[item].iterrows():
        region_id = datasetDict[item].loc[index,'ID']
        jhi_value = datasetDict[item].loc[index,name]
        if jhi_value == 0:
            a = datasetDict[item].loc[index,name1]
            b = datasetDict[item].loc[index,name2]
            #print(name1 + ": " + str(a) + "\n" + name2 + ": " + str(b))
            if (a != b): #checar se algum valor é diferente
                print(name1 + ": " + str(a) + "\n" + name2 + ": " + str(b) + "\n" + "linha" + ": " + str(index))

RO
AC
AM
RR
PA
AP
TO
MA
PI
CE
RN
PB
PE
AL
SE
BA
MG
ES
RJ
SP
PR
SC
RS
MS
MT
GO
DF


In [58]:
datasetDict['RN']

,Unnamed: 0,Município_x,UF,Idade,Total_Estado,Total_Homem,Total_Mulher,Urbana_Total,Urbana_Homem,Urbana_Mulher,Rural_Total,Rural_Homem,Rural_Mulher,editMun,ID,Município_y
5355,5355,Acari (RN),RN,Total,2888,1454,1434,2321,1159,1162,567,295,272,acari,1076,Acari
5360,5360,Açu (RN),RN,Total,14975,7514,7461,11074,5484,5590,3901,2030,1871,acu,1077,Açu
5365,5365,Afonso Bezerra (RN),RN,Total,2979,1515,1464,1602,782,820,1377,733,644,afonsobezerra,1078,Afonso Bezerra
5370,5370,Água Nova (RN),RN,Total,898,460,438,586,295,291,312,165,147,aguanova,1079,Água Nova
5375,5375,Alexandria (RN),RN,Total,3363,1705,1658,2245,1123,1122,1118,582,536,alexandria,1080,Alexandria
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6155,6155,Várzea (RN),RN,Total,1422,717,705,1193,595,598,229,122,107,varzea,1238,Várzea
6160,6160,Venha-Ver (RN),RN,Total,1133,592,541,372,193,179,761,399,362,venha-ver,1239,Venha-Ver
6165,6165,Vera Cruz (RN),RN,Total,2956,1476,1480,1254,626,628,1702,850,852,veracruz,1240,Vera Cruz
6170,6170,Viçosa (RN),RN,Total,440,213,227,415,201,214,25,12,13,vicosa,1241,Viçosa


In [59]:
datasetDict['RN'].loc[6170]

Unnamed: 0              6170
Município_x      Viçosa (RN)
UF                        RN
Idade                  Total
Total_Estado             440
Total_Homem              213
Total_Mulher             227
Urbana_Total             415
Urbana_Homem             201
Urbana_Mulher            214
Rural_Total               25
Rural_Homem               12
Rural_Mulher              13
editMun               vicosa
ID                      1241
Município_y           Viçosa
Name: 6170, dtype: object